In [5]:
# data_retrieval_and_export.ipynb
import asyncio
import pandas as pd
from lib.data_utils import fetch_and_prepare_data

# Fetch and prepare data
sample_size = 10_000

dataset_exists = False
try:
    data_df = pd.read_csv('data_samples/paragraph-questions-10k.csv')
    dataset_exists = True
    print('Dataset already exists.')
except FileNotFoundError:
    data_df = await fetch_and_prepare_data(sample_size)
    data_df.to_csv('data_samples/paragraph-questions-10k.csv', index=False)

# Display the first few rows of the DataFrame
data_df.head()


Dataset already exists.


,question_id,question_text,author_id,paragraph_id,paragraph_text,author_model,author_username
0,140,Why did historian Ronald Hutton refer to these...,1,975,The Early Neolithic people of Britain placed f...,mistralai/Mixtral-8x7B-Instruct-v0.1,None
1,141,Why did historian Ronald Hutton refer to these...,1,975,The Early Neolithic people of Britain placed f...,mistralai/Mixtral-8x7B-Instruct-v0.1,None
2,142,"What belief system, involving the veneration o...",1,975,The Early Neolithic people of Britain placed f...,mistralai/Mixtral-8x7B-Instruct-v0.1,None
3,462,How many goals did Lionel Messi score for Arge...,1,713132,Top goalscorer for Argentina in a calendar yea...,mistralai/Mixtral-8x7B-Instruct-v0.1,None
4,463,Which record did Lionel Messi set in 2012 by s...,1,713132,Top goalscorer for Argentina in a calendar yea...,mistralai/Mixtral-8x7B-Instruct-v0.1,None
